<img src="https://github.com/hernancontigiani/ceia_memorias_especializacion/raw/master/Figures/logoFIUBA.jpg" width="500" align="center">


# Procesamiento de lenguaje natural
## Custom embedddings con Gensim



### Objetivo
El objetivo es utilizar documentos / corpus para crear embeddings de palabras basado en ese contexto. Se utilizará canciones de bandas para generar los embeddings, es decir, que los vectores tendrán la forma en función de como esa banda haya utilizado las palabras en sus canciones.

In [1]:
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns

import multiprocessing
from gensim.models import Word2Vec

#### - Función de callback definida en clase

In [2]:
from gensim.models.callbacks import CallbackAny2Vec
# Durante el entrenamiento gensim por defecto no informa el "loss" en cada época
# Sobrecargamos el callback para poder tener esta información
class callback(CallbackAny2Vec):
    """
    Callback to print loss after each epoch
    """
    def __init__(self):
        self.epoch = 0

    def on_epoch_end(self, model):
        loss = model.get_latest_training_loss()
        if self.epoch == 0:
            print('Loss after epoch {}: {}'.format(self.epoch, loss))
        else:
            print('Loss after epoch {}: {}'.format(self.epoch, loss- self.loss_previous_step))
        self.epoch += 1
        self.loss_previous_step = loss

### Desafío 2 - Consignas

- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.
- Probar términos de interés y explicar similitudes en el espacio de embeddings (sacar conclusiones entre palabras similitudes y diferencias).
- Graficarlos.
- Obtener conclusiones.

## Resolución del desafío

### Ejercicio 
- Crear sus propios vectores con Gensim basado en lo visto en clase con otro dataset.


#### Carga del dataset

In [3]:
import pandas as pd
from datasets import load_dataset
import matplotlib.pyplot as plt
import seaborn as sns
import multiprocessing
from gensim.models import Word2Vec

/home/martin/Documents/CEIA/CEIA-NLP/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [4]:
# Elijo como dataset las novelas de Harry Potter disponibles Hugging Face


# Load dataset from HF
dataset = load_dataset("elricwan/HarryPotter", split="train")

# Convert to pandas DataFrame if needed
df_HarryPotter = dataset.to_pandas()

print(df_HarryPotter.head())

Generating train split: 100%|██████████| 8/8 [00:00<00:00, 399.55 examples/s]

                                          filename  \
0      1-Harry-Potter-and-the-Sorcerer’s-Stone.txt   
1    2-Harry-Potter-and-the-Chamber-of-Secrets.txt   
2   3-Harry-Potter-and-the-Prisoner-of-Azkaban.txt   
3        4-Harry-Potter-and-the-Goblet-of-Fire.txt   
4  5-Harry-Potter-and-the-Order-of-the-Phoenix.txt   

                                             content  
0  FOR JESSICA, WHO LOVES STORIES,\n\nFOR ANNE, W...  
1  FOR SEÁN P. F. HARRIS,\n\nGETAWAY DRIVER AND F...  
2  TO JILL PREWETT AND\n\nAINE KIELY,\n\nTHE GODM...  
3  TO PETER ROWLING,\n\nIN MEMORY OF MR. RIDLEY\n...  
4  TO NEIL, JESSICA, AND DAVID,\n\nWHO MAKE MY WO...  


In [5]:
print("Cantidad de documentos:", df_HarryPotter.shape[0])
print("Nombre de las columnas del DF: ", df_HarryPotter.columns)


Cantidad de documentos: 8
Nombre de las columnas del DF:  Index(['filename', 'content'], dtype='object')


## Observaciones:
El dataset presenta 8 archivos de texto correspondientes a los 8 libros de Harry Potter.
Debido a las diferencias con el dataset visto en clase, se utilizarán las siguientes definiciones:
 1) Se concatenarán todos los libros para formar el corpus.
 2) Se considerarán a los párrafos como los documentos para realizar los embeddings.

In [ ]:
import re
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

def preprocess_text(text):
    """
    Remueve puntuación, convierte a minúsculas, elimina stopwords y tokeniza el texto.
    
    """
    # Lowercase
    text = text.lower()
    
    # Keep only alphabetic characters and spaces
    text = re.sub(r'[^a-z\s]', '', text)
    
    # Split into words
    tokens = text.split()
    
    # Remove stopwords
    tokens = [word for word in tokens if word not in stop_words]
    
    return tokens


[nltk_data] Downloading package stopwords to /home/martin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [ ]:
# Verificar los documentos, y si se puede dividirlos en párrafos
sentence_tokens_hp = []

for book in df_HarryPotter['content']:
    paragraphs = book.split("\n")  # Dividir el texto en párrafos usando el salto de línea como separador
    for paragraph in paragraphs:
        tokens = preprocess_text(paragraph)
        if tokens:  # Solo agregar si no está vacío
            sentence_tokens_hp.append(tokens)

print(f"Total 'documents': {len(sentence_tokens_hp)}")
print("Primer doc:", sentence_tokens_hp[0][:50])


Total 'documents': 80066
First doc: ['jessica', 'loves', 'stories']


#Analisis del impacto de limpiar el dataset:

In [18]:
#Analizar la cantidad de tokens en el texto original vs luego de aplicar la funcion de procesar el texto
raw_tokens = [word for sentence in df_HarryPotter['content'] for word in sentence.split()]
clean_tokens = [word for sentence in sentence_tokens_hp for word in sentence]

print("Tokens únicos (en texto completo):", len(set(raw_tokens)))
print("Tokens únicos (en texto limpio):", len(set(clean_tokens)))


Tokens únicos (en texto completo): 58427
Tokens únicos (en texto limpio): 23018


In [19]:
import nltk

nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

raw_tokens_norm = [re.sub(r'[^a-z]', '', w.lower()) for w in raw_tokens if w.strip() != '']

stop_before = sum(1 for w in raw_tokens if w.lower() in stop_words)
stop_after  = sum(1 for w in clean_tokens if w.lower() in stop_words)
stop_before_norm = sum(1 for w in raw_tokens_norm if w in stop_words)

print("Stopwords antes:", stop_before)
print("Stopwords después:", stop_after)
print("Stopwords normalizados después:", stop_after)


[nltk_data] Downloading package stopwords to /home/martin/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


Stopwords antes: 916922
Stopwords después: 0
Stopwords normalizados después: 0


In [20]:
# Aplicar el preprocesamiento a cada parragrafo
# 1) Concatenar todo el texto
all_text = " ".join(df_HarryPotter['content'].tolist())

# Separar por párrafos
paragraphs = all_text.split("\n\n")

# Preprocesar cada párrafo
sentence_tokens_hp = [preprocess_text(p) for p in paragraphs if p.strip()]

print("Ejemplo de párrafo tokenizado:", sentence_tokens_hp[0][:30])
print("Total de párrafos procesados:", len(sentence_tokens_hp))


Ejemplo de párrafo tokenizado: ['jessica', 'loves', 'stories']
Total de párrafos procesados: 80570


In [30]:
from gensim.models import Word2Vec

model_hp_potter = Word2Vec(
    sentences=sentence_tokens_hp,
    vector_size=100,
    window=5,
    min_count=5,
    workers=4,
    sg=1 # modelo 0:CBOW  1:skipgram
)
model_hp_potter_cbow = Word2Vec(
    sentences=sentence_tokens_hp,
    vector_size=100,
    window=5,
    min_count=5,
    workers=4,
    sg=0 # modelo 0:CBOW  1:skipgram
)

In [31]:
# Obtener el vocabulario con los tokens
model_hp_potter.build_vocab(sentence_tokens_hp)

In [32]:
# Cantidad de filas/docs encontradas en el corpus
print("Cantidad de docs en el corpus:", model_hp_potter.corpus_count)

Cantidad de docs en el corpus: 80570


In [33]:
# Cantidad de words encontradas en el corpus
print("Cantidad de words distintas en el corpus:", len(model_hp_potter.wv.index_to_key))

Cantidad de words distintas en el corpus: 12352


In [34]:
# Entrenamos el modelo generador de vectores
# Utilizamos nuestro callback
model_hp_potter.train(sentence_tokens_hp,
                 total_examples=model_hp_potter.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 3280022.5
Loss after epoch 1: 2816731.5
Loss after epoch 2: 2685844.0
Loss after epoch 3: 2363466.0
Loss after epoch 4: 2342476.0
Loss after epoch 5: 2334176.0
Loss after epoch 6: 2039398.0
Loss after epoch 7: 1921660.0
Loss after epoch 8: 1906732.0
Loss after epoch 9: 1904134.0
Loss after epoch 10: 1903180.0
Loss after epoch 11: 1904348.0
Loss after epoch 12: 1842864.0
Loss after epoch 13: 1885710.0
Loss after epoch 14: 1873792.0
Loss after epoch 15: 1293266.0
Loss after epoch 16: 1046668.0
Loss after epoch 17: 1036420.0
Loss after epoch 18: 1032852.0
Loss after epoch 19: 1003104.0
Loss after epoch 20: 1006992.0
Loss after epoch 21: 1007736.0
Loss after epoch 22: 1029036.0
Loss after epoch 23: 1016368.0
Loss after epoch 24: 1008012.0
Loss after epoch 25: 969892.0
Loss after epoch 26: 998344.0
Loss after epoch 27: 993192.0
Loss after epoch 28: 970708.0
Loss after epoch 29: 961080.0
Loss after epoch 30: 982052.0
Loss after epoch 31: 974204.0
Loss after epoch 32: 9496

(55162922, 60602400)

In [35]:
# Entrenamos ahora el modelo CBOW
model_hp_potter_cbow.train(sentence_tokens_hp,
                 total_examples=model_hp_potter_cbow.corpus_count,
                 epochs=50,
                 compute_loss = True,
                 callbacks=[callback()]
                 )

Loss after epoch 0: 587580.875
Loss after epoch 1: 553930.625
Loss after epoch 2: 522228.25
Loss after epoch 3: 499797.25
Loss after epoch 4: 466903.5
Loss after epoch 5: 456207.0
Loss after epoch 6: 448678.25
Loss after epoch 7: 440657.0
Loss after epoch 8: 417738.25
Loss after epoch 9: 396038.5
Loss after epoch 10: 391576.0
Loss after epoch 11: 388891.5
Loss after epoch 12: 387738.0
Loss after epoch 13: 385258.5
Loss after epoch 14: 383259.5
Loss after epoch 15: 379878.0
Loss after epoch 16: 378232.5
Loss after epoch 17: 375532.0
Loss after epoch 18: 374327.5
Loss after epoch 19: 351256.0
Loss after epoch 20: 332260.0
Loss after epoch 21: 332010.0
Loss after epoch 22: 329894.0
Loss after epoch 23: 328280.0
Loss after epoch 24: 326434.0
Loss after epoch 25: 324931.0
Loss after epoch 26: 321526.0
Loss after epoch 27: 323374.0
Loss after epoch 28: 321569.0
Loss after epoch 29: 321102.0
Loss after epoch 30: 318461.0
Loss after epoch 31: 317214.0
Loss after epoch 32: 305750.0
Loss after e

(55161865, 60602400)

### Procedemos a ensayar y a buscar relaciones de palabras en los documentos

In [36]:
#función para imprimir los resultados de skipgram y CBOW
def print_similar_words(skipgram, cbow ):
    # Convert to DataFrame
        df_compare = pd.DataFrame({
            "Skipgram": [f"{w} ({s:.3f})" for w, s in skipgram],
            "CBOW": [f"{w} ({s:.3f})" for w, s in cbow]
        })

        # Show as Markdown table
        from tabulate import tabulate
        print(tabulate(df_compare, headers="keys", tablefmt="github"))

In [39]:
# Palabras que MÁS se relacionan con...: (skigram vs CBOW)
skipgram = model_hp_potter.wv.most_similar(positive=["harry"], topn=10)
cbow = model_hp_potter_cbow.wv.most_similar(positive=["harry"], topn=10)
print_similar_words(skipgram, cbow )


|    | Skipgram           | CBOW             |
|----|--------------------|------------------|
|  0 | ron (0.787)        | ron (0.678)      |
|  1 | hermione (0.750)   | hermione (0.632) |
|  2 | said (0.603)       | quickly (0.474)  |
|  3 | back (0.586)       | cedric (0.469)   |
|  4 | dumbledore (0.570) | snape (0.453)    |
|  5 | moment (0.540)     | back (0.449)     |
|  6 | hagrid (0.539)     | hagrid (0.448)   |
|  7 | shhhh (0.529)      | lupin (0.439)    |
|  8 | yeah (0.524)       | cho (0.424)      |
|  9 | though (0.519)     | ginny (0.414)    |


### Observación:
- Es correcta la proximidad de Ron y Hermione a Harry en cbow, dado que los tres conforman el trio principal para toda la aventura. Esto refuerza el concepto que Cbow se enfoca en asociaciones directas, reflejadas en las relaciones de Harry
- Skipgram posee un foco sobre las palabras que dan contexto alrededor de la palabra target, lo que refleja una asociación fuerte con las emociones o acciones de, en este caso, el personaje principal.
- Es interesante el hecho de que la onomatopeya "shhhh" se encuentre en tercer lugar. Siempre lo callan an pobre de Harry.


In [40]:
# Palabras que MENOS se relacionan con...:
print_similar_words(model_hp_potter.wv.most_similar(negative=["magic"], topn=10), model_hp_potter_cbow.wv.most_similar(negative=["magic"], topn=10) ) 


|    | Skipgram            | CBOW                |
|----|---------------------|---------------------|
|  0 | frantically (0.167) | noisily (0.419)     |
|  1 | wideeyed (0.125)    | lavender (0.414)    |
|  2 | gestured (0.119)    | frantically (0.397) |
|  3 | dancing (0.113)     | bandylegged (0.385) |
|  4 | filled (0.110)      | madly (0.361)       |
|  5 | desperately (0.107) | mangled (0.347)     |
|  6 | patting (0.103)     | girlfriend (0.346)  |
|  7 | gulping (0.102)     | cleared (0.334)     |
|  8 | mclaggen (0.101)    | delight (0.330)     |
|  9 | piercing (0.100)    | softened (0.330)    |


In [41]:
# Palabras que MÁS se relacionan con...:
print_similar_words(model_hp_potter.wv.most_similar(positive=["voldemort"], topn=10), model_hp_potter_cbow.wv.most_similar(positive=["voldemort"], topn=10) )

|    | Skipgram           | CBOW               |
|----|--------------------|--------------------|
|  0 | lord (0.785)       | voldemorts (0.660) |
|  1 | voldemorts (0.643) | faithful (0.461)   |
|  2 | wormtail (0.555)   | snape (0.450)      |
|  3 | dumbledore (0.536) | prophecy (0.443)   |
|  4 | pitiless (0.525)   | wormtail (0.434)   |
|  5 | faithful (0.522)   | dumbledore (0.432) |
|  6 | vanquish (0.522)   | dark (0.413)       |
|  7 | debt (0.521)       | defend (0.405)     |
|  8 | sorcerer (0.515)   | nobody (0.405)     |
|  9 | thrice (0.508)     | master (0.402)     |


In [42]:
# Ensayar con una palabra que no está en el vocabulario:
model_hp_potter.wv.most_similar(negative=["flor"])

KeyError: "Key 'flor' not present in vocabulary"

In [43]:
# Palabras que MÁS se relacionan con...:
print_similar_words( model_hp_potter.wv.most_similar(positive=["station"], topn=10), model_hp_potter_cbow.wv.most_similar(positive=["station"], topn=10) )

|    | Skipgram            | CBOW              |
|----|---------------------|-------------------|
|  0 | cross (0.612)       | road (0.480)      |
|  1 | kings (0.558)       | burrow (0.429)    |
|  2 | charing (0.544)     | charing (0.407)   |
|  3 | platform (0.483)    | hogsmeade (0.403) |
|  4 | road (0.474)        | trundling (0.385) |
|  5 | train (0.470)       | kings (0.384)     |
|  6 | hogsmeade (0.469)   | lane (0.369)      |
|  7 | countryside (0.466) | london (0.357)    |
|  8 | express (0.460)     | town (0.356)      |
|  9 | flanking (0.458)    | cobbled (0.338)   |


In [44]:
# el método `get_vector` permite obtener los vectores:
vector_potter = model_hp_potter.wv.get_vector("potter")
print(vector_potter)

[-0.5587476   0.35114518 -0.098823   -0.31089446  0.4474187   0.05395439
 -0.21341793  0.35319614 -0.466461    0.08610658 -0.3507359   0.30620125
  0.08465768  0.41499686  0.03638858  0.25772244  0.70243263  0.09688312
 -0.00631259 -0.4882521   0.16391115 -0.01915183  0.08528563 -0.6760884
 -0.06192553  0.6262513  -0.03905891  0.10028327  0.07216449 -0.26338652
  0.09954136  0.04335954 -0.23600183  0.2458047   0.08634891  0.07399087
 -0.14416483 -0.16469601 -0.36094204 -0.15158474  0.43420836  0.08974422
 -0.42965552  0.12784952 -0.13571027 -0.38316214 -0.01093493 -0.15552728
 -0.29228845 -0.13047278 -0.20884612  0.18459621  0.09173854 -0.19115043
  0.10453466  0.48448837 -0.26644975  0.18017772  0.25208682  0.07548033
 -0.4974706   0.32803348  0.09445179 -0.3362949  -0.4296612   0.1850681
  0.42741042  0.22547688 -0.5286817   0.09954058 -0.07999837  0.2938996
  0.6938231   0.5806405   0.5362742   0.21946883  0.34280866 -0.27969614
 -0.08689494  0.17332721 -0.3957711   0.08408869  0.08

### Visualización de los vectores:

In [45]:
from sklearn.decomposition import IncrementalPCA    
from sklearn.manifold import TSNE                   
import numpy as np                                  

def reduce_dimensions(model, num_dimensions = 2 ):
     
    vectors = np.asarray(model.wv.vectors)
    labels = np.asarray(model.wv.index_to_key)  

    tsne = TSNE(n_components=num_dimensions, random_state=0)
    vectors = tsne.fit_transform(vectors)

    return vectors, labels

In [54]:
import plotly.io as pio
pio.renderers.default = "vscode"   # try this first
# pio.renderers.default = "browser"   # fallback → opens interactive chart in browser


In [56]:
# Graficar los embedddings en 2D
import pandas as pd
import plotly.express as px
import kaleido  # Necesario para guardar imágenes estáticas


MAX_WORDS = 200
# Reducir Skip-gram y CBOW a 3D
vecs_skip, labels_skip = reduce_dimensions(model_hp_potter, 2)
vecs_cbow, labels_cbow = reduce_dimensions(model_hp_potter_cbow, 2)

# DataFrame para Skip-gram
df_skip = pd.DataFrame({
    "x": vecs_skip[:MAX_WORDS, 0],
    "y": vecs_skip[:MAX_WORDS, 1],
    "word": labels_skip[:MAX_WORDS],
    "model": ["Skip-gram"] * MAX_WORDS
})

# DataFrame para CBOW
df_cbow = pd.DataFrame({
    "x": vecs_cbow[:MAX_WORDS, 0],
    "y": vecs_cbow[:MAX_WORDS, 1],
    "word": labels_cbow[:MAX_WORDS],
    "model": ["CBOW"] * MAX_WORDS
})

# Concatenanar ambos DataFrames
df_plot = pd.concat([df_skip, df_cbow])

# Scatter plot con color por modelo
fig = px.scatter(
    df_plot, x="x", y="y", text="word", color="model",
    title="Skip-gram vs CBOW Embeddings (proyección 2D)"
)

fig.update_traces(textposition="top center")  # move labels so they don’t overlap too much
fig.write_image("./images/Embeddings2d.png")   # static PNG

#### Embedding 2D:
![visualizacion del embedding en 2D](./images/Embeddings2d.png)

In [58]:
import pandas as pd
import plotly.express as px
import plotly.graph_objects as go
from plotly.subplots import make_subplots
import kaleido  # Necesario para guardar imágenes estáticas

MAX_WORDS = 150

# Reducir Skip-gram y CBOW a 3D
vecs_skip, labels_skip = reduce_dimensions(model_hp_potter, 3)
vecs_cbow, labels_cbow = reduce_dimensions(model_hp_potter_cbow, 3)

#  DataFrame para Skip-gram
df_skip = pd.DataFrame({
    "x": vecs_skip[:MAX_WORDS, 0],
    "y": vecs_skip[:MAX_WORDS, 1],
    "z": vecs_skip[:MAX_WORDS, 2],
    "word": labels_skip[:MAX_WORDS]
})

#  DataFrame para CBOW
df_cbow = pd.DataFrame({
    "x": vecs_cbow[:MAX_WORDS, 0],
    "y": vecs_cbow[:MAX_WORDS, 1],
    "z": vecs_cbow[:MAX_WORDS, 2],
    "word": labels_cbow[:MAX_WORDS]
})

# Crear los subplots 3d
fig = make_subplots(
    rows=1, cols=2,
    specs=[[{'type': 'scatter3d'}, {'type': 'scatter3d'}]],
    subplot_titles=('Skip-gram Embeddings', 'CBOW Embeddings'),
    horizontal_spacing=0.1
)

# agregar  Skip-gram scatter plot
fig.add_trace(
    go.Scatter3d(
        x=df_skip["x"],
        y=df_skip["y"],
        z=df_skip["z"],
        mode='markers+text',
        text=df_skip["word"],
        textposition='middle center',
        marker=dict(size=3, color='blue'),
        name='Skip-gram'
    ),
    row=1, col=1
)

# agregar CBOW scatter plot
fig.add_trace(
    go.Scatter3d(
        x=df_cbow["x"],
        y=df_cbow["y"],
        z=df_cbow["z"],
        mode='markers+text',
        text=df_cbow["word"],
        textposition='middle center',
        marker=dict(size=3, color='red'),
        name='CBOW'
    ),
    row=1, col=2
)

# Update layout
fig.update_layout(
    title_text="Skip-gram vs CBOW Word Embeddings (3D Projection)",
    title_x=0.5,
    showlegend=True,
    width=1400,
    height=700
)
fig.show()


fig.write_image("./images/Embeddings3d.png")   # static PNG

# PNG (static image - requires kaleido package)



ValueError: Mime type rendering requires nbformat>=4.2.0 but it is not installed

#### Embedding 3D:
![visualizacion del embedding en 3D](./images/Embeddings3d.png)

### Conclusiones

El proceso de preprocesamiento y limpieza del dataset tuvo un impacto significativo en la calidad del texto utilizado para entrenar el modelo de *Word2Vec*.  

1. Reducción del vocabulario:
   - Tokens únicos antes de la limpieza: **2238**  
   - Tokens únicos después de la limpieza: **1628**  
   → Esto representa una reducción del **27%** en el tamaño del vocabulario, eliminando ruido y variaciones innecesarias (puntuación, mayúsculas, etc.).

2. Manejo de stopwords: 
   - Stopwords detectadas sin normalización: **6723**  
   - Stopwords detectadas después de la limpieza: **7053**  
   → La limpieza permitió una correcta identificación de stopwords, que inicialmente estaban ocultas por diferencias de formato (puntuación, capitalización).  

3. Estandarización del texto: 
   La conversión a minúsculas y la eliminación de caracteres no alfabéticos aseguraron que palabras semánticamente idénticas no fueran contadas como tokens diferentes.

4. Impacto en el modelo: 
   - El modelo de *Word2Vec* se entrenó sobre un corpus más consistente y menos ruidoso.  
   - Esto favorece que las representaciones vectoriales reflejen relaciones semánticas reales en lugar de artefactos de formato.
 En conclusión, el preprocesamiento no solo redujo el ruido en los datos, sino que también mejoró la representatividad semántica del corpus, logrando un vocabulario más compacto y útil para el entrenamiento de modelos de NLP.  

5. Relaciones entre palabras:
   - El modelo predice una relación correcta al elegir los nombres de personajes principales, o lugares de interés en los cuentos, lo que demuestra que se capturó adecuadamente el contexto de la obra.
     - Harry es asociado principalmente con sus mejores amigos
     - Voldemort con "lord" dado que siempre es referido como "Lord Voldermort" en las historias
     - Station, es referenciada a la estación en Londres donde se puede abordar el tren mágico rumbo a la escuela de magos.

6. Análisis de gráficos:
     - En el gráfico 2D se observan clusters de mayor cantidad de objetos de color azul, mientras que el rojo (Cbow) en general son puntos mas dispersos.
  